In [ ]:
class InputDataset(Dataset):
    def __init__(self, dir, output_size=(128, 128)):
        
        self.dir = dir
        self.output_size = output_size
        self.image_names = []
        
        for image_name in os.listdir(self.dir):
          _, image_extension = os.path.splitext(image_name)
          if image_extension == '.jpg':
            self.image_names.append(image_name)
          elif image_extension == '.JPG':
            self.image_names.append(image_name)
          elif image_extension == '.PNG':
            self.image_names.append(image_name)
          elif image_extension == '.png':
            self.image_names.append(image_name)  
         
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, i):
        cur_image_name = self.image_names[i] 
        
        # Load images
        input_image = Image.open(os.path.join(self.dir, cur_image_name))
        
        # Random crop
        i, j, h, w = transforms.RandomCrop.get_params(
            input_image, 
            output_size=self.output_size
            )
        input_image = TF.crop(input_image, i, j, h, w)
        
        #To tensor
        input_image = TF.to_tensor(input_image)

        input_dict = {
            'input': input_image
        }              
        return input_dict




class ShadingDataset(Dataset):
    def __init__(self, dir, output_size=(128, 128)):
    
        self.dir = dir
        self.output_size = output_size
        self.image_names = []
        
        for image_name in os.listdir(self.dir):
          _, image_extension = os.path.splitext(image_name)
          if image_extension == '.jpg':
            self.image_names.append(image_name)
          elif image_extension == '.JPG':
            self.image_names.append(image_name)
          elif image_extension == '.PNG':
            self.image_names.append(image_name)
          elif image_extension == '.png':
            self.image_names.append(image_name)
        

    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, i):
         
        cur_image_name = self.image_names[i] 
        # Load images
        shade_image = Image.open(os.path.join(self.dir, cur_image_name))
        # Random crop
        i, j, h, w = transforms.RandomCrop.get_params(
            shade_image, 
           output_size=self.output_size
          )
        shade_image = TF.crop(shade_image, i, j, h, w)

        #To tensor
        shade_image = TF.to_tensor(shade_image)

        shading_dict = {
            'shading_gt':shade_image
        }              

        return shading_dict

In [ ]:
########## Hyperparameter ##########
base_lr = 1e-5
batch_size = 4 #64
epochs = 50
EV = np.zeros((2,1))
mu = np.zeros((99,1))
PC = np.zeros((99,2))
i = 0
with open('/content/drive/MyDrive/VMG Candidate Project/CSV_Files/PC.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      PC[i:,] = row 
      i=i+1
PC = torch.tensor(PC).to(device)
i = 0
with open('/content/drive/MyDrive/VMG Candidate Project/CSV_Files/EV.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      EV[i:,] = row 
      i=i+1
EV = torch.tensor(EV)
i = 0
with open('/content/drive/MyDrive/VMG Candidate Project/CSV_Files/mu.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      mu[i:,] = row 
      i=i+1
mu = torch.tensor(mu).to(device)

illumA = np.zeros((1,1,33))
i = 0
with open('/content/drive/MyDrive/VMG Candidate Project/CSV_Files/illumA.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      illumA[i:,] = row 
      i=i+1
illumA = torch.tensor(illumA)
illumANorm = illumA/torch.sum(illumA)
illumANorm = illumANorm.to(device)

illumDNorm = np.zeros((22,33))
i = 0
with open('/content/drive/MyDrive/VMG Candidate Project/CSV_Files/illumD.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      illumDNorm[i:,] = row 
      i=i+1

illumDNorm = torch.tensor(illumDNorm)
illumDNorm = torch.transpose(illumDNorm,0,1)
illumDNorm = illumDNorm.reshape(1,1,33,22)

for i in range(22):
  illumDNorm[:,:,:,i] = illumDNorm[:,:,:,i]/torch.sum(illumDNorm[:,:,:,i]) 

illumDNorm = illumDNorm.to(device)
illFNorm= np.zeros((1,1,33,12))
for i in range(1,13):
  fileName = '/content/drive/MyDrive/VMG Candidate Project/CSV_Files/illF' + str(i) + '.csv'
  with open(fileName, 'r') as file:
      reader = csv.reader(file)
      for row in reader:
        illFNorm[:,:,:,i-1] = row
        
illFNorm = torch.tensor(illFNorm).to(device)

Tmatrix = np.zeros((128,128,9))
TmatTemp = np.zeros((128,128))
for i in range(1,10):
  fileName = '/content/drive/MyDrive/VMG Candidate Project/CSV_Files/Tmatrix' + str(i) + '.csv'
  with open(fileName, 'r') as file:
      reader = csv.reader(file)
      j=0
      for row in reader:
        TmatTemp[:,j] = row
        j = j + 1
      Tmatrix[:,:,i-1] = TmatTemp
Tmatrix = torch.tensor(Tmatrix).to(device)

Newskincolor = np.zeros((256,256,33))
NewskinTemp = np.zeros((256,256))
for i in range(1,33):
  fileName = '/content/drive/MyDrive/VMG Candidate Project/CSV_Files/Newskincolor' + str(i) + '.csv'
  with open(fileName, 'r') as file:
      reader = csv.reader(file)
      j=0
      for row in reader:
        NewskinTemp[:,j] = row
        j = j + 1
      Newskincolor[:,:,i-1] = NewskinTemp
Newskincolor = torch.tensor(Newskincolor).to(device)

muim = torch.tensor([129.1863, 104.7624, 93.5940])
muim = muim.reshape(1,1,3)
wavelength = 33
lossweights = torch.tensor([1e-3,1e-4,1e-5,1e-5])
display_epoch = 1  
display_scale = 1
train_display = True
test_display = True
num_workers = 0 #4
shuffle = False
save_flag = True
save_epoch = 1
model_path = '/content/drive/MyDrive/VMG Candidate Project/Model_Param' 
model_name = 'BioFaceNet.pth'

options = {
    'base_lr': base_lr,
    'batch_size': batch_size,
    'epochs': epochs,
    'mu': mu,
    'muim': muim,
    'PC':PC,
    'EV':EV,
    'wavelength':wavelength,
    'lossweights': lossweights,
    'display_epoch': display_epoch,
    'display_scale': display_scale,
    'train_display': train_display,
    'illumANorm': illumANorm,
    'illumDNorm': illumDNorm,
    'illumFNorm': illFNorm,
    'NewSkinColor':Newskincolor,
    'Tmatrix': Tmatrix,
    'test_display': test_display,
    'num_workers': num_workers,
    'shuffle': shuffle,
    'save_flag': save_flag, 
    'save_epoch': save_epoch,
    'model_path': model_path,
    'model_name': model_name
}

if not os.path.isdir(model_path):
    print('Create new dir.')
    os.mkdir(model_path)